# Creating thickness maps from GeoTOP

In this example we will retrieve GeoTOP from the OpenDAP server for an area around Delft and 
visualize the thickness of several geological layers and lithologies to gain
a quick insight in how the subsurface in this area is built-up.

First, we import the GeoTop class from GeoST and retrieve GeoTOP data within the selected
bounding box.

In [ ]:
from geost.bro import GeoTop

# Load the GeoTOP model for the Delft area
geotop = GeoTop.from_opendap(bbox=(80000, 442000, 89000, 450000))

print(geotop)

As you can see, geotop_delft is now an instance of [`GeoTop`](../api_reference/bro_geotop.rst) 
and provides a number of methods to make further selections and exports. 
We will make use of the GeoTop.get_thickness method.

We'd like to know the total thickness of clay layers in the area. Let's first check the
GeoTop lithology encoding, which can be accessed from the [`Lithology`](../api_reference/geotop_selection.rst)
class or through downloading the reference list from the [GeoTOP page on the BRO website](https://basisregistratieondergrond.nl/inhoud-bro/registratieobjecten/modellen/geotop-gtm/).
The code for 'Clay' in GeoTOP is 2, so we will use that in the get_thickness method as follows:

In [ ]:
# Import the Lithology class to access GeoTOP lithological types
from geost.bro import Lithology

# Get the thickness of clay layers in the GeoTOP model and plot it
thickness_clay = geotop.get_thickness(geotop["lithok"] == Lithology.clay)
thickness_clay.plot.imshow(vmin=0, vmax=20)

This shows the total thickness of all clay layers combined in the GeoTOP model. However,
we are only interested in Holocene clay layers. The stratigraphic units defined in the
[`StratGeotop`](../api_reference/geotop_selection.rst) class can be used to refine the selection. Alternatively, 
you can also reference the [GeoTOP documentation](https://basisregistratieondergrond.nl/inhoud-bro/registratieobjecten/modellen/geotop-gtm/) and use the stratigraphic units numbering 
directly. We will use the `StratGeotop` class in this example:

In [ ]:
# Import the StratGeotop class to access GeoTOP stratigraphic units
from geost.bro import StratGeotop

# Get the total thickness of clay in the Holocene stratigraphic units
thickness_clay_holocene = geotop.get_thickness(
    (geotop["lithok"] == Lithology.clay) & (geotop["strat"].isin(StratGeotop.holocene))
)
thickness_clay_holocene.plot.imshow(vmin=0, vmax=20)

We can also get the thickness of, say, basal peat. The code for organic material is 1 and
the stratigraphic unit we're looking for is the Formatie van Nieuwkoop - Basisveen Laag, 
which is encoded as 1130.

In [ ]:
# Get the thickness of basal peat
thickness_basal_peat = geotop.get_thickness(
    (geotop["lithok"] == Lithology.organic)
    & (geotop["strat"] == StratGeotop.select_units("NIBA"))
)
thickness_basal_peat.plot.imshow()

Since the results are Xarray DataArrays, we can export them using rioxarray to geotiffs
for sharing and viewing the results in GIS software for example.

In [ ]:
# Export thickness_basal_peat to a GeoTIFF using rioxarray
thickness_basal_peat.rio.to_raster("thickness_basal_peat.tif")